In [1]:
import pandas as pd
import xgboost as xgb

In [2]:
normalized_train = pd.read_csv('../result/all_beta_normalized_train_035.csv')

normalized_test = pd.read_csv('../result/all_beta_normalized_test_035.csv')

normalized_train_c = normalized_train[normalized_train['Unnamed: 0'].isin(normalized_test['Unnamed: 0'])]

In [3]:
X_train = normalized_train_c.iloc[:, 1::2]

x = []

for i in range(445):
    X = X_train.iloc[:, i]

    X = X.values.flatten().tolist()

    x.append(X)

y = [ (0 if i < 47 else 1)  for i in range(445)]

In [4]:
from sklearn.model_selection import cross_val_score

# 將數據轉換為DMatrix格式
dmatrix = xgb.DMatrix(x, label=y)

# 定義模型參數
params = {
    'objective': 'binary:logistic',  # 二分類問題
    'eval_metric': 'logloss',  # 評估指標
    'max_depth': 3,
    'learning_rate': 1
}

# 建立模型
model = xgb.XGBClassifier(**params)

# 進行交叉驗證，這裡使用 5-fold 交叉驗證
cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')

# 印出每次交叉驗證的準確度
print("各次交叉驗證的準確度:", cv_scores)

# 計算平均準確度
average_accuracy = cv_scores.mean()
print("平均交叉驗證準確度:", average_accuracy)


各次交叉驗證的準確度: [0.98876404 0.97752809 0.96629213 0.98876404 0.98876404]
平均交叉驗證準確度: 0.9820224719101123


In [21]:
from itertools import combinations

target_labels = ['cg03520644','cg05047401','cg06355129','cg07790615','cg09106903']

# 三個一組
combinations_list = list(combinations(target_labels, 3))

acc=[]
# 所有可能的組合
for combination in combinations_list:
    #print(combination)
    d1_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[0]]
    d2_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[1]]
    d3_values = normalized_train.loc[normalized_train['Unnamed: 0']==combination[2]]
   
    d1 = d1_values.iloc[:, 1::2]
    d2 = d2_values.iloc[:, 1::2]
    d3 = d3_values.iloc[:, 1::2]

    d1 = d1.values.flatten().tolist()
    d2 = d2.values.flatten().tolist()
    d3 = d3.values.flatten().tolist()

    x = []

    for i in range(445):
        x.append([d1[i],d2[i],d3[i]])
        
    # 將數據轉換為DMatrix格式
    dmatrix = xgb.DMatrix(x, label=y)

    # 定義模型參數
    params = {
        'objective': 'binary:logistic',  # 二分類問題
        'eval_metric': 'logloss',  # 評估指標
        'max_depth': 3,
        'learning_rate': 1
    }

    # 建立模型
    model = xgb.XGBClassifier(**params)

    # 進行交叉驗證，這裡使用 5-fold 交叉驗證
    cv_scores = cross_val_score(model, x, y, cv=5, scoring='accuracy')

    # 印出每次交叉驗證的準確度
    #print("各次交叉驗證的準確度:", cv_scores)

    # 計算平均準確度
    average_accuracy = cv_scores.mean()
    #print("平均交叉驗證準確度:", average_accuracy)
    acc.append([combination[0],combination[1], combination[2], cv_scores[0],cv_scores[1],cv_scores[2],cv_scores[3],cv_scores[4], average_accuracy])


In [23]:
df=pd.DataFrame(acc, columns=['ID1', 'ID2', 'ID3', 'fold_scores1','fold_scores2','fold_scores3','fold_scores4','fold_scores5', 'average_accuracy'])
df.to_csv("../result/xgb.csv", index=False)